In [1]:
import numpy as np
import torch
from transformers import GPTJForCausalLM, AutoTokenizer, AutoModel, GPT2LMHeadModel, AutoModelForCausalLM

import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device = ", device)

/home/dmpowell/.conda/envs/pytorch-gpu-2.0/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device =  cuda


In [2]:
MODEL_NAME = "EleutherAI/gpt-j-6B" # gpt2-xl / "EleutherAI/gpt-j-6B" / "databricks/dolly-v1-6b"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, low_cpu_mem_usage=True).to(device)# model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v1-6b", low_cpu_mem_usage=True).to(device)
# model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).cuda()
model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v1-6b", low_cpu_mem_usage=True).to(device)

In [6]:
def generate_text(texts, model):
    if type(texts) != list:
        texts = [texts]
    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.eos_token
    encoding = tokenizer(texts, padding=True, return_tensors='pt').to(device)
    with torch.no_grad():
        generated_ids = model.generate(**encoding, do_sample=True, temperature=0.1, max_new_tokens=100)

        generated_texts = tokenizer.batch_decode(
            generated_ids, skip_special_tokens=True
        )
        
    return(generated_texts)

person = "Miles Davis"
texts = [f"{person} has won"]

texts = [f"List 5 interesting facts about {person}. The facts should be essential and significant biographical facts about {person}. List each fact on its own line and write each as a complete sentence.\n"]

for t in generate_text(texts, model): print(t)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


List 5 interesting facts about Miles Davis. The facts should be essential and significant biographical facts about Miles Davis. List each fact on its own line and write each as a complete sentence.

1. Miles Davis was born in 1926 in Alton, Illinois.
2. He was the youngest of three children.
3. His father was a jazz musician and his mother was a homemaker.
4. He began playing the trumpet at the age of six.
5. He was a member of the Charlie Parker Quintet in the 1940s.
6. He was a member of the Miles Davis Quintet in the 1950s.
7. He was a member


In [5]:
def generate_text(texts, model):
    if type(texts) != list:
        texts = [texts]
    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.eos_token
    encoding = tokenizer(texts, padding=True, return_tensors='pt').to(device)
    with torch.no_grad():
        generated_ids = model.generate(**encoding, 
                                       do_sample=True, 
                                       temperature=0.7, 
                                       max_new_tokens=10,
                                       num_return_sequences = 5
                                      )

        generated_texts = tokenizer.batch_decode(
            generated_ids, skip_special_tokens=True
        )
        
    return(generated_texts)


subject = "Isaac Newton"

prompts = [
    f"{subject} is",
    f"{subject} is considered",
    f"{subject} has won",
    f"{subject} has been named",
    f"{subject} was selected",
    f"{subject} failed to",
    f"{subject} has been featured in",
    f"{subject} was born on",
    f"{subject}'s father was",
    f"{subject}'s mother was",
    f"growing up, {subject}'s home life was",
    f"{subject} received the",
    f"{subject} early on showed a strong aptitude for",
    f"{subject} earned their",
    f"{subject} worked in",
    f"{subject} invented the",
    f"{subject} married",
    f"{subject} founded",
    f"{subject} investigated",
    f"{subject} read many",
    f"{subject} wrote",
    f"{subject} studied",
    f"{subject} bought ",
    f"{subject} invests",
    f"{subject} eats",
    f"{subject} drinks",
    f"{subject} best friends",
    f"{subject} is an active supporter of",
    f"{subject} took a stance on the issue of",
    f"{subject} signed",
    f"{subject} debuted in",
    f"{subject} acted alongside",
    f"{subject} starred in",
    f"{subject} appeared in",
    f"{subject} appeared opposite",
    f"{subject} was featured in",
    f"{subject} died of",
    f"{subject} was inspired by",
    f"{subject} struggled with",
    f"{subject} was guided by",
    f"{subject} grew up",
    f"{subject} moved to",
    f"{subject} was educated at",
]

generate_text(prompts[0:2], model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Isaac Newton is the inventor of calculus, which he himself called �',
 'Isaac Newton is the most famous and most influential scientist in the history',
 'Isaac Newton is widely regarded as one of the greatest scientists and mathematic',
 'Isaac Newton is one of the most influential scientists of all time.',
 'Isaac Newton is one of the most important scientists of all time and',
 'Isaac Newton is considered one of the greatest scientists of all time. He',
 'Isaac Newton is considered to be the father of calculus, and this is',
 'Isaac Newton is considered to be a founding father of modern science. He',
 'Isaac Newton is considered to be one of the greatest\nphilosophers',
 'Isaac Newton is considered to be one of the most influential scientists in history']

In [5]:
from evaluate import load
bertscore = load("bertscore")

generations = generate_text(prompts, model)
gens_per = len(generations) // len(prompts)
out = []

for i in range(len(prompts)):
    gens = generations[i*gens_per:i*gens_per+gens_per]
    gens = [g[len(prompts[i]):] for g in gens]
    predictions = gens
    references = ["astronomy"]*len(gens)
    results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
    val = max(results["recall"])

    resdict = dict()
    resdict["prompt"] = prompts[i]
    resdict["gens"] = gens
    resdict["val"] = val
    
    out.append(resdict)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 42.00 MiB (GPU 0; 15.78 GiB total capacity; 14.56 GiB already allocated; 28.19 MiB free; 14.79 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [68]:
out_sorted = sorted(out, key=lambda d: d['val'])
out_sorted[:10]

[{'prompt': 'Isaac Newton debuted in',
  'gens': [' 1802, the same year as the first issue',
   ' the third episode of the first season, The Secret',
   ' the UK in the early 19th century as a',
   ' the pages of Uncanny X-Men in 1963',
   ' the U.S. in 1704, but'],
  'val': 0.6077552437782288},
 {'prompt': 'Isaac Newton has been featured in',
  'gens': [' more than 30 articles in the prestigious Times Higher Education',
   ' more than 30 books, including books published by both',
   ' more than 250,000 books, articles, and',
   ' a number of books and articles over the years.',
   ' the New York Times and the Financial Times, and'],
  'val': 0.6101351380348206},
 {'prompt': 'Isaac Newton was educated at',
  'gens': [' Cambridge and taught at the Royal Academy of Arts in',
   ' Blenheim Palace, Oxford University and the Royal',
   " St. Paul's, Oxford, and Trinity College",
   ' the University of Cambridge, but it was not until',
   ' the Royal Academy, Cambridge, which is where he'],


In [45]:
i = 0
generations[i*3:i*3+3]

['Mother Theresa is a famous example, where people were not allowed to',
 'Mother Theresa is a great example of what a good Catholic mother looks',
 'Mother Theresa is a Catholic nun who lived in a nunnery for']

In [7]:
from evaluate import load
bertscore = load("bertscore")
predictions = gens
references = ["basketball"]*len(gens)
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")

IndexError: list index out of range

In [33]:
for i in range(len(prompt_list)):
    print(gens[i], results["recall"][i])

 the greatest player in the history of the NBA. 0.6966029405593872
 by many to be the greatest basketball player of all 0.8198080658912659
 five NBA titles, but he’s never 0.749808132648468
 the NBA's Most Valuable Player for the second 0.694441556930542
 with the No. 1 overall pick in the 2003 0.6216462850570679
 make the NBA All-Star Game for the first 0.692640483379364
 a lot of commercials, but this one is a 0.6400294303894043
 June 24, 1984 in Akron, Ohio. He 0.5305962562561035
 a basketball player, but he was a football player 0.8613694906234741
 a drug addict and his father was a drug addict 0.5931428670883179
 anything but normal. His mother, Gloria, was 0.5750566124916077
 most votes in the NBA's All-Star Game 0.6799660325050354
 basketball. He was a natural athlete, and he 0.9124909043312073
 first win of the season, and the first of 0.6045515537261963
 the mailroom at the Akron Beacon Journal for a 0.6194360852241516
 “Lebron” nickname. It 0.6275466680526733
 Savannah Brinso

' a heart attack on June 23, 2009. He'

In [31]:
results

{'precision': [0.5720751881599426,
  0.5724977254867554,
  0.5799227952957153,
  0.5837574601173401,
  0.552601158618927,
  0.5778065323829651,
  0.5763013362884521,
  0.5418870449066162,
  0.6511839628219604,
  0.559006929397583,
  0.5949299335479736,
  0.5695910453796387,
  0.6581129431724548,
  0.5512027740478516,
  0.5531152486801147,
  0.609715461730957,
  0.5456223487854004,
  0.544346272945404,
  0.5551638603210449,
  0.5973663330078125,
  0.5476133227348328,
  0.6006280779838562,
  0.5169603824615479,
  0.5537319779396057,
  0.5792847871780396,
  0.5879291296005249,
  0.5896029472351074,
  0.5688070058822632,
  0.6017779111862183,
  0.4813586175441742,
  0.6025204062461853,
  0.5699297189712524,
  0.54963219165802,
  0.54963219165802,
  0.5619406700134277,
  0.5953479409217834,
  0.5436211824417114,
  0.6495181918144226,
  0.5618728995323181,
  0.6103910207748413],
 'recall': [0.6966029405593872,
  0.8198080658912659,
  0.749808132648468,
  0.694441556930542,
  0.62164628505706